In [15]:
output_folder = r'P:\watmodel\CWATM\Regions\Bhima\Input\settings\output'

In [37]:
from netCDF4 import Dataset, num2date
import numpy.ma as ma
import numpy as np
import xlsxwriter
import os

cellArea = Dataset(output_folder +'/'+ 'cellArea_totalend.nc', 'r').variables['cellArea_totalend'][:,:]
cellAreaC = ma.compressed(cellArea)
cellAreaSum = np.sum(ma.compressed(cellAreaC))

dir_list = os.listdir(output_folder)

workbook = xlsxwriter.Workbook('CWatM_output_variables.xlsx')
worksheet = workbook.add_worksheet('🤍')

column=0
for file in dir_list:
    daily = False
    time = True
    if file[-3:]=='.nc':
        var = file[:-3]
        print(var)
        
        # variables have the timestep appened on the variable name, ex. _totalend, _monthavg, _monthend
        # except for daily variables
        if file[:-3].split('_')[-1] == daily:
            var = var[:-6]
            daily = True
            
        if daily:   
            netcdf = Dataset(output_folder +'/'+ var +'_daily.nc', 'r') #.variables[var]
        else:
            netcdf = Dataset(output_folder +'/'+ var +'.nc', 'r')
        
        if 'time' in netcdf.variables.keys():
            print(netcdf.variables['time'].units)
            if 'Months since' in netcdf.variables['time'].units
                times = netcdf.variables['time']
                dates_simulation = [datetime.datetime(1901,1,1)+datetime.timedelta(seconds = 365.25/12*24.0*3600.0*float(val)) for val in times]
            else:
                dates_simulation = num2date(netcdf.variables['time'][:], units=netcdf.variables['time'].units)
        else: time = False
            
        # unit of varible

        netcdf_var = netcdf.variables[var]
        var_unit = netcdf_var.units
        print(var_unit)
        
        if cellArea.shape == netcdf_var.shape:
        
            worksheet.write(0, column, 'date')
            worksheet.write(0, column+1, var)
            if not time:
                    if var_unit == 'm':
                        daily_basinSum = np.sum(np.multiply(ma.compressed(netcdf_var[:,:]), cellAreaC))

                    elif var_unit == 'm2' or var_unit == 'm3':
                        daily_basinSum = np.sum(ma.compressed(netcdf_var[:,:]))

                    elif var_unit == 'm3s':
                        daily_basinSum = np.sum(np.multiply(ma.compressed(netcdf_var[:,:]), 60*60*24))

                    elif var_unit == '%':
                        daily_basinSum = np.sum(np.multiply(ma.compressed(netcdf_var[:,:]), cellAreaC))
                        daily_basinSum /= cellAreaSum

                    else: # soon combined with previous
                        daily_basinSum = np.sum(np.multiply(ma.compressed(netcdf_var[:,:]), cellAreaC))
                        daily_basinSum /= cellAreaSum

                    worksheet.write(d+1, column, 'Without time')
                    worksheet.write(d+1, column+1, daily_basinSum)

            else:   
                for d in range(len(dates_simulation)):

                    if var_unit == 'm':
                        daily_basinSum = np.sum(np.multiply(ma.compressed(netcdf_var[d,:,:]), cellAreaC))

                    elif var_unit == 'm2' or var_unit == 'm3':
                        daily_basinSum = np.sum(ma.compressed(netcdf_var[d,:,:]))

                    elif var_unit == 'm3s':
                        daily_basinSum = np.sum(np.multiply(ma.compressed(netcdf_var[d,:,:]), 60*60*24))

                    elif var_unit == '%':
                        daily_basinSum = np.sum(np.multiply(ma.compressed(netcdf_var[d,:,:]), cellAreaC))
                        daily_basinSum /= cellAreaSum

                    else: # soon combined with previous
                        daily_basinSum = np.sum(np.multiply(ma.compressed(netcdf_var[d,:,:]), cellAreaC))
                        daily_basinSum /= cellAreaSum

                    worksheet.write(d+1, column, str(dates_simulation[d]))
                    worksheet.write(d+1, column+1, daily_basinSum)
                    row += 1  

            column += 2


SyntaxError: expected ':' (1032369326.py, line 37)

In [ ]:

import xlsxwriter
import numpy.ma as ma

# WB = {Inputs:  [ ['Rain', [rain_value_day0, ..., rain_value_dayLast]] ],
#       Outputs: [ ['Evapotranspiration (soil)', [ET_value_day0, ..., ET_value_dayLast]] 
#                  ['Channel evaporation', [ChannelE_value_day0, ..., ChannelE_value_dayLast]], ... ]}

keys = [i[-1] for i in Vars]
WB = {key: [] for key in keys}

cellArea = Dataset(output_folder +'/'+ 'cellArea_totalend.nc', 'r').variables['cellArea_totalend'][:,:]
cellAreaC = ma.compressed(cellArea)

cellAreaSum = np.sum(ma.compressed(cellAreaC))

#limitAbstraction = True
print("Loading the following variables:")
counter = 1 
if Excel:
    workbook = xlsxwriter.Workbook('waterCycle'+str(int(latitude))+'.xlsx')
    worksheet = workbook.add_worksheet('0')
    
for var in Vars:

    row = 1
    if Excel:
        worksheet.write(0, counter, str(var[0]))
    print(var[0])
    nc_filename = output_folder +'/'+ var[0] + '_daily.nc'
    temp = Dataset(nc_filename, 'r').variables[var[0]][:,:,:]
    Daily_basinSum = []
    
    for d in range(len(Dates_simulation)):
        
        if var[2] == 'M':
            daily_basinSum = np.sum(np.multiply(ma.compressed(temp[d,:,:]), cellAreaC))
            if limitAbstraction == True and var[0]=='unmetDemand': 
                daily_basinSum *=0
        elif var[2] == 'M3':
            daily_basinSum = np.sum(ma.compressed(temp[d,:,:]))
        elif var[2] == 'MS':
            daily_basinSum = np.sum(np.multiply(ma.compressed(temp[d,:,:]), 60*60*24))
        else:
            #All cells hold the same total-value for the basin
            daily_basinSum = ma.compressed(temp[d,:,:])[0]
            
        Daily_basinSum.append(daily_basinSum)

        if Excel:
            if counter == 1:
                worksheet.write(row, 0, str(Dates_simulation[d]))
            worksheet.write(row, counter, daily_basinSum)
            row += 1     
    
    WB[var[-1]].append([var[1], np.array(Daily_basinSum)])
    counter += 1
    
if Excel:
    row = 1
    worksheet.write(0, counter, 'discharge')
    for d in range(len(Dates_simulation)):
        worksheet.write(row, counter, FLOWS_simulated[0][d]*86400)
        row += 1
    workbook.close()